In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

data = pd.read_excel("C:\\Users\\HoaHT15.Onsiter\\Desktop\\New folder\\Auto_Causal\\0707.xlsx", sheet_name = None)
data = pd.concat(data.values(), ignore_index=True)
# Drop rows with missing values
data.dropna(inplace=True)
# Convert 'SAGYO_UNYO_DATE' column to datetime format
data['SAGYO_UNYO_DATE'] = pd.to_datetime(data['SAGYO_UNYO_DATE'], format='%Y%m%d')
# Convert 'SGY_TM' column to datetime format
data['SGY_TM'] = pd.to_datetime(data['SGY_TM'], format='%Y/%m/%d %H:%M:%S')


In [43]:
input_startDate = pd.to_datetime('2022-06-11')
input_endDate = pd.to_datetime('2022-06-13')
num_days = (input_endDate - input_startDate).days

compare_startDate = pd.to_datetime('2022-07-11')
compare_endDate = pd.to_datetime('2022-07-13')

df_origin = data[(data['SAGYO_UNYO_DATE'] >= input_startDate) & (data['SAGYO_UNYO_DATE'] <= input_endDate)]
column_names = ['CATE','NIOKURI_NM','CAPA', 'TANKA','NIUKE_NM','KATAMEI','FD_BUNRUI']

# W-1 
w_minus1_startDate = input_startDate - timedelta(days=7)
w_minus1_endDate = w_minus1_startDate + timedelta(days=num_days)
print("Week-1 Start Date:", w_minus1_startDate.date())
print("Week-1 End Date:", w_minus1_endDate.date())
# W + 1
w_plus1_startDate = input_startDate + timedelta(days=7)
w_plus1_endDate = w_plus1_startDate + timedelta(days=num_days)
print("Week+1 Start Date:", w_plus1_startDate.date())
print("Week+1 End Date:", w_plus1_endDate.date())
# M - 1
m_minus1_startDate = input_startDate - relativedelta(months=1)
m_minus1_endDate = m_minus1_startDate + timedelta(days=num_days)
print("Month-1 Start Date:", m_minus1_startDate.date())
print("Month-1 End Date:", m_minus1_endDate.date())
# M + 1
m_plus1_startDate = input_startDate + relativedelta(months=1)
m_plus1_endDate = m_plus1_startDate + timedelta(days=num_days)
print("Month+1 Start Date:", m_plus1_startDate.date())
print("Month+1 End Date:", m_plus1_endDate.date())
# Quarter - 1 
q_minus1_startDate = input_startDate - relativedelta(months=3)
q_minus1_endDate = q_minus1_startDate + timedelta(days=num_days)
print("Quarter - 1 Start Date:", q_minus1_startDate.date())
print("Quarter - 1 End Date:", q_minus1_endDate.date())
# Quarter + 1
q_plus1_startDate = input_startDate + relativedelta(months=3)
q_plus1_endDate = q_plus1_startDate + timedelta(days=num_days)
print("Quarter + 1 Start Date:", q_plus1_startDate.date())
print("Quarter + 1 End Date:", q_plus1_endDate.date())
# Year - 1
y_minus1_startDate = input_startDate - relativedelta(years=1)
y_minus1_endDate = y_minus1_startDate + timedelta(days=num_days)
print("Year - 1 Start Date:", y_minus1_startDate.date())
print("Year - 1 End Date:", y_minus1_endDate.date())
# Year + 1
y_plus1_startDate = input_startDate + relativedelta(years=1)
y_plus1_endDate = y_plus1_startDate + timedelta(days=num_days)
print("Year + 1 Start Date:", y_plus1_startDate.date())
print("Year + 1 End Date:", y_plus1_endDate.date())

Week-1 Start Date: 2022-06-04
Week-1 End Date: 2022-06-06
Week+1 Start Date: 2022-06-18
Week+1 End Date: 2022-06-20
Month-1 Start Date: 2022-05-11
Month-1 End Date: 2022-05-13
Month+1 Start Date: 2022-07-11
Month+1 End Date: 2022-07-13
Quarter - 1 Start Date: 2022-03-11
Quarter - 1 End Date: 2022-03-13
Quarter + 1 Start Date: 2022-09-11
Quarter + 1 End Date: 2022-09-13
Year - 1 Start Date: 2021-06-11
Year - 1 End Date: 2021-06-13
Year + 1 Start Date: 2023-06-11
Year + 1 End Date: 2023-06-13


In [44]:
def calculation(variable_name, startDate, endDate):
    df_compare = data[(data['SAGYO_UNYO_DATE'] >= startDate) & (data['SAGYO_UNYO_DATE'] <= endDate)]
    if len(df_compare) <= 0:
        return 
    sum_df1 = df_origin.groupby(variable_name, as_index=False)['SGY_JSK_QTY'].sum()
    # Compare with range date
    sum_df2 = df_compare.groupby(variable_name, as_index=False)['SGY_JSK_QTY'].sum()
    df = pd.merge(sum_df1, sum_df2, on=variable_name)
    # df = df
    df.columns = ['TYPE']  +  ['SGY_JSK_QTY_x'] +  ['SGY_JSK_QTY_y']
    df['CHANGE_AMOUNT'] = df['SGY_JSK_QTY_x'] - df['SGY_JSK_QTY_y']
    df['CHANGE(%)'] = (df['CHANGE_AMOUNT']/df['SGY_JSK_QTY_x'] * 100).round(2)
    df['COLUMN_NAME'] = variable_name
    df = df.sort_values('CHANGE(%)').query("CHANGE_AMOUNT < 0").head(1)
    return df
def call_calculation(startDate, endDate):
    df_full = pd.DataFrame()
    for col_name in column_names:
        df = calculation(col_name, startDate, endDate)
        df_full = pd.concat([df_full, df])

    df_full.reset_index(drop=True, inplace=True)
    # add an index column starting from 1
    df_full.index = df_full.index + 1
    return df_full

df_intput_compare = call_calculation(compare_startDate, compare_endDate)
print('compare')
print(df_intput_compare)
# week - 1 & w + 1
dfw_minus1 = call_calculation(w_minus1_startDate, w_minus1_endDate)
print('week -1')
print(dfw_minus1)
dfw_plus1 = call_calculation(w_plus1_startDate, w_plus1_endDate)
print('week + 1')
print(dfw_plus1)
# month - 1 & M +1
dfm_minus1 = call_calculation(m_minus1_startDate, m_minus1_endDate)
print('month -1')
print(dfm_minus1)
dfm_plus1 = call_calculation(m_plus1_startDate, m_plus1_endDate)
print('month + 1')
print(dfm_plus1)
# Quarter - 1 & Q + 1
dfq_minus1 = call_calculation(q_minus1_startDate, q_minus1_endDate)
print('quarter -1')
print(dfq_minus1)
dfq_plus1 = call_calculation(q_plus1_startDate, q_plus1_endDate)
print('quarter + 1')
print(dfq_plus1)
# Year - 1 & Year + 1
dfy_minus1 = call_calculation(y_minus1_startDate, y_minus1_endDate)
print('year -1')
print(dfy_minus1)
dfy_plus1 = call_calculation(y_plus1_startDate, y_plus1_endDate)
print('year + 1')
print(dfy_plus1)

compare
        TYPE  SGY_JSK_QTY_x  SGY_JSK_QTY_y  CHANGE_AMOUNT  CHANGE(%)  \
1         情報           6398          24323         -17925    -280.17   
2  株式会社　浅沼商会              1             70            -69   -6900.00   
3    0.00002             29           6673          -6644  -22910.34   
4        786              1            208           -207  -20700.00   
5    枚方店(客注)              1             43            -42   -4200.00   
6   LR03R/4S             27           6643          -6616  -24503.70   
7   63705151              1             89            -88   -8800.00   

  COLUMN_NAME  
1        CATE  
2  NIOKURI_NM  
3        CAPA  
4       TANKA  
5    NIUKE_NM  
6     KATAMEI  
7   FD_BUNRUI  
week -1
             TYPE  SGY_JSK_QTY_x  SGY_JSK_QTY_y  CHANGE_AMOUNT  CHANGE(%)  \
1             理美容           1294           2869          -1575    -121.72   
2        シロカ　株式会社              4            104           -100   -2500.00   
3        0.002652              3            304 